In [1]:
import os
import sys
import time
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

from src.data import load_annotation, load_data
from src.utils.train import train_test_split

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

OSError: [WinError 1455] The paging file is too small for this operation to complete. Error loading "C:\Users\Jiajun\Desktop\download-project\.download-project\lib\site-packages\torch\lib\caffe2_detectron_ops_gpu.dll" or one of its dependencies.

In [13]:
# config
DATAFRAME_PATH = "../data/raw/data_frames"
ANNOTATION_PATH = "../data/processed/Annotation.csv"
CATEGORY = "Urination"
THRESHOLD = 0.3

FEATURE_NAMES = ['Min', 'Max', 'Median', 'Mean', 'LogVariance', 'LinearTrend']
SOURCES = ['TotalWeight', 'WaterDistance', 'AudioDelay', 'RadarSum', 'AudioDelay2', 'AudioDelay4']
CATEGORY = 'Urination'

ANNOTATIONS = load_annotation.get_annotation(ANNOTATION_PATH)
USER_IDS = load_annotation.get_complete_ids(ANNOTATION_PATH, CATEGORY)
TRAIN_IDS, TEST_IDS = train_test_split(USER_IDS)

print ("Training on {} cases, {} ...".format(len(TRAIN_IDS), TRAIN_IDS[:5]))
print ("Testing  on {} cases, {} ...".format(len(TEST_IDS), TEST_IDS[:5]))

Training on 117 cases, [2021, 2005, 1918, 2027, 2015] ...
Testing  on 30 cases, [1802, 1806, 1830, 1831, 1836] ...


In [ ]:
rf_train_config = {
    'USE_IDS': TRAIN_IDS,
    'ANNOTATION_PATH': ANNOTATION_PATH,
    'FEATURE_NAMES': FEATURE_NAMES,
    'SOURCES': SOURCES,
    'CATEGORY': CATEGORY
}

rf_test_config = {
    'USE_IDS': TEST_IDS,
    'ANNOTATION_PATH': ANNOTATION_PATH,
    'FEATURE_NAMES': FEATURE_NAMES,
    'SOURCES': SOURCES,
    'CATEGORY': CATEGORY
}

dataset = {}
dataset['train'] = load_data.RandomForestDataset(rf_train_config)
dataset['test'] = load_data.RandomForestDataset(rf_test_config)

In [ ]:
train_x, train_y = dataset['train'].get_all_features_and_labels()
test_x, test_y = dataset['test'].get_all_features_and_labels()

In [ ]:
def classification_result(model, testX, testY):
    testYPredProb = model.predict_proba(testX)
    testYPred = (testYPredProb[:, 1] > 0.5).astype(int)
    print (classification_report(testY, testYPred))

def variable_importance(trainX, model):
    plt.bar(x = range(trainX.shape[1]), height = model.feature_importances_)
    xticks_pos = np.arange(trainX.shape[1])
    plt.xticks(xticks_pos, trainX.columns, rotation=45, ha = 'right')
    pass

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, plot_roc_curve

rf = RandomForestClassifier(n_estimators = 30, max_features = 15)
rf.fit(train_x, train_y)

In [ ]:
print ("Training on {} cases, {} ...".format(len(TRAIN_IDS), TRAIN_IDS[:5]))
print ("Testing  on {} cases, {} ...".format(len(TEST_IDS), TEST_IDS[:5]))

In [ ]:
# variable importance
plt.figure(figsize = (12, 6))
variable_importance(train_x, rf)

In [ ]:
import pickle
import time

timestr = time.strftime("%Y%m%d-%H%M%S")
with open(f"../randomforest-{timestr}.pkl", "wb") as f:
    pickle.dump(rf, f)

In [2]:
import pickle

with open("../randomforest-20210108-032342.pkl", "rb") as f:
    rf = pickle.load(f)

In [11]:
DATAFRAME_PATH = "../data/raw/data_frames"
ANNOTATION_PATH = "../data/processed/Annotation.csv"
CATEGORY = "Urination"
THRESHOLD = 0.3

FEATURE_NAMES = ['Min', 'Max', 'Median', 'Mean', 'LogVariance', 'LinearTrend']
SOURCES = ['TotalWeight', 'WaterDistance', 'AudioDelay', 'RadarSum', 'AudioDelay2', 'AudioDelay4']
CATEGORY = 'Urination'

eval_config = {
    'USE_ID': 1919,
    'SOURCES': SOURCES,
    'FEATURE_NAMES': FEATURE_NAMES,
    'CATEGORY': CATEGORY,
    'MODEL': rf,
    'ANNOTATION_PATH': ANNOTATION_PATH
}

evaluation = Evaluation(eval_config)
X = evaluation.get_predicted_regions()
#X = evaluation.get_feature_and_label()

Updating user : 1919


NameError: name 'predictions' is not defined

In [9]:
def booleanToRegions(labels, start_time):
    res = []
    start = 1
    while (start < len(labels)):
        while (start < len(labels) and labels[start] == 0):
            start += 1
        if (start < len(labels) - 1):
            end = start + 1
            while (end < len(labels) and labels[end] == 1):
                end += 1
            res.append([start_time + start, start_time + end - 1])
            start = end    
    return res

class Evaluation:
    def __init__(self, eval_config):
        self.use_id = eval_config['USE_ID']
        self.sources = eval_config['SOURCES']
        self.category = eval_config['CATEGORY']
        self.feature_names = eval_config['FEATURE_NAMES']
        self.model = eval_config['MODEL']
        self.annotation_path = eval_config['ANNOTATION_PATH']
    
    def get_feature_and_label(self):
        config = {
            'USE_IDS': [self.use_id],
            'ANNOTATION_PATH': self.annotation_path,
            'FEATURE_NAMES': self.feature_names,
            'SOURCES': self.sources,
            'CATEGORY': self.category
        }
        return load_data.RandomForestDataset(config).get_all_features_and_labels()
    
    def get_annotated_regions(self):
        annotations = load_annotation.get_annotation(self.annotation_path)
        regions = annotations[use_i]
        defecate_regions = [region[:2] for region in regions if region[2] == 'Defecation']
        urinate_regions = [region[:2] for region in regions if region[2] == 'Urination']
        return urinate_regions, defecate_regions
    
    def get_predicted_regions(self, threshold = 0.3):
        feature, label = self.get_feature_and_label()
        prediction = self.model.predict_proba(feature)
        prediction = (prediction[:, 1] > threshold).astype(int)
        regions = booleanToRegions(prediction, st)
        return regions

def PlotWithAnnotationPrediction(use_i, model, category, threshold = 0.3):
    urinate_regions, defecate_regions = getAnnotatedRegions(use_i)
    predicted_regions = getPredictedRegions(use_i, model, threshold)
    totalweight = getTotalWeightsz(use_i)

    fig, ax = plt.subplots(2, 1, figsize = (15, 2*2), sharex = True)
    ax[0].plot(totalweight)
    ax[0].title.set_text('{} : total weight'.format(use_i))
    ax[0].set_ylim(totalweight.median() - 0.5, totalweight.median() + 0.5)
    ax[1].plot(totalweight)
    ax[1].title.set_text('{} : total weight'.format(use_i))
    ax[1].set_ylim(totalweight.median() - 0.5, totalweight.median() + 0.5)
    if category == "Urination":
        regions = urinate_regions
    else:
        regions = defecate_regions
        
    for region in regions:
        ax[0].axvspan(region[0], region[1] + 1, alpha=0.5, color='gold')
    for region in predicted_regions:
        ax[1].axvspan(region[0], region[1] + 1, alpha=0.5, color='red')
    plt.tight_layout()
    plt.savefig("evaluation/defecation_{}.jpg".format(use_i))
    plt.show()

In [ ]:
PlotWithAnnotationPrediction(1919, rf, "Urination")